In [9]:
# Melhor alternativa para resumir
import whisper
from openai import OpenAI

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.luhn import LuhnSummarizer

In [2]:
# Função pra escrever o texto do Whisper em um TXT
def add_to_textfile(text_file, text):
  text = text.encode('utf-8')
  normalized_text = text.decode('utf-8')
  
  with open(text_file, 'w', encoding='utf-8') as f:
    f.write(normalized_text)
  return  normalized_text

# Função pra consultar o chatGPT
def get_completion(prompt:str,api_key:str,model="gpt-3.5-turbo"):
  client = OpenAI(api_key = api_key)
  message = [{"role": "user", "content": prompt}]
  response = client.chat.completions.create(model=model,messages=message)
  return response.choices[0].message["content"]

In [3]:
try:
    model = whisper.load_model("medium")
    result = model.transcribe(r"audio.mp3")
    bulk_text = add_to_textfile(r"content\bulktText.txt",result['text'])
except Exception as error:
    print(error)

c:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\whisper\transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [4]:
# Criando um Resumo do Texto

parser = PlaintextParser.from_string(bulk_text,Tokenizer('portuguese'))
sumarizer = LuhnSummarizer()
summary = sumarizer(parser.document,40)
summary

(<Sentence: Bom, hoje eu queria falar desse assunto que é um caso que está rolando aqui no Japão, a mídia está acompanhando isso daí, não se fala de outra coisa no Japi, o bafafá que só vendo.>,
 <Sentence: Na verdade, toda essa treta começou no final de agosto, mas foi se desenrolando, foi se desenrolando de uma forma escrota que assim foi, é um bagulho que...>,
 <Sentence: A gente estava vendo só a pontinha do iceberg lá em agosto e aí o bagulho foi se desenrolando da maneira mais escrota possível e está muito interessante, mano, de acompanhar.>,
 <Sentence: Então talvez alguns de vocês já tenham até visto falar, porque tudo isso começou no final de agosto quando um japonês foi procurar a polícia lá em Nagoya, Nagoya que é uma das grandes cidades aqui do Japão, acho que é a quarta maior cidade aqui do Japão, inclusive tem muito brasileiro que mora lá, né?>,
 <Sentence: Porque na região de Aichi lá tem muita fábrica e tal, e aí um japonês foi procurar a polícia lá em Nagoya dizendo qu

In [6]:
prompt = f"Faça um resumo de 20 linhas como se fosse um estudante que assistiu a uma palestra: {result['text']}"
# Basta inserir sua API-KEY do ChatGPT da OpenAI 
response = get_completion(prompt,"API-KEY")
print(response)

In [8]:
# Formatando o resumo do texto
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.probability import FreqDist
from nltk.tokenize.treebank import TreebankWordDetokenizer

# Baixar os recursos necessários do NLTK
nltk.download('punkt')
nltk.download('stopwords')

def text_summarizer(text, ratio=0.2):
    # Tokenizar o texto em frases
    sentences = sent_tokenize(text)

    # Tokenizar as palavras e remover stopwords
    stop_words = set(stopwords.words("portuguese"))
    words = word_tokenize(text, language='portuguese')
    filtered_words = [word.lower() for word in words if word.isalnum() and word.lower() not in stop_words]

    # Calcular a frequência das palavras
    fdist = FreqDist(filtered_words)

    # Ordenar as frases com base na soma das frequências das palavras
    ranking = {}
    for i, sentence in enumerate(sentences):
        for word in word_tokenize(sentence.lower(), language='portuguese'):
            if word in fdist:
                if i not in ranking:
                    ranking[i] = fdist[word]
                else:
                    ranking[i] += fdist[word]

    # Ordenar as frases pelo ranking
    sorted_sentences = sorted(ranking, key=ranking.get, reverse=True)

    # Selecionar as principais frases com base na taxa especificada
    num_sentences = int(len(sentences) * ratio)
    selected_sentences = sorted(sorted_sentences[:num_sentences])

    # Reorganizar as frases na ordem original
    summary = TreebankWordDetokenizer().detokenize([sentences[i] for i in sorted(selected_sentences)])

    return summary

resumo = text_summarizer(bulk_text)
bulk_text = add_to_textfile(r"content\summary.txt",resumo)
print(resumo)

Então talvez alguns de vocês já tenham até visto falar, porque tudo isso começou no final de agosto quando um japonês foi procurar a polícia lá em Nagoya, Nagoya que é uma das grandes cidades aqui do Japão, acho que é a quarta maior cidade aqui do Japão, inclusive tem muito brasileiro que mora lá, né? Porque na região de Aichi lá tem muita fábrica e tal, e aí um japonês foi procurar a polícia lá em Nagoya dizendo que estava preocupado com uma menina chamada Miku que ele tinha conhecido pelo Line, né? E aí esse cara falou lá na polícia que ele costumava conversar todo dia pelo Line com essa mina, até que do nada ela parou de responder e alguém mandou uma mensagem do celular dela para ele dizendo que a Miku foi sequestrada, tá ligado? Até aí já tá esquisito pra caralho, né? Tipo a mina foi sequestrada e aí a pessoa que sequestrou ela mandou uma mensagem pelo celular dela para um cara random que ela nem conhecia pessoalmente avisando, né? Porque não é possível um bagulho desse, então era 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Windows\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Windows\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
